In [2]:
from pathlib import Path
import shutil

In [3]:
# Define the base directory
dataset_dir = Path("/mnt/ssd2/xin/repo/DART/Liebherr_Product")

# Define the images directory and duplicates directory using Path objects
image_dir = dataset_dir / "images"
lvis_output_dir = dataset_dir / "images_predicted" / "lvis"
lp_output_dir = dataset_dir / "images_predicted" / "lp"
gt_output_dir = dataset_dir / "images_predicted" / "gt"
yolo_dir = dataset_dir / "yolo"

with open(yolo_dir / "test_dedup_gpt_8020.txt", "r") as f:
    image_list = f.read().splitlines()

# objs = sorted(list(set([i.split('/')[-2] for i in image_list])))

In [ ]:
# from ultralytics import YOLO
# lvis_weight_path = "runs/detect/lvis/weights/best.pt"
# yolo = YOLO(lvis_weight_path)

# images_detected_lvis = []

# for img in image_list:
#     input_path = dataset_dir / img
#     result = yolo(input_path,verbose=False)[0]
#     if result.boxes.shape[0] != 0: # only save images with detected objects
#         images_detected_lvis.append(img)
#         output_path = lvis_output_dir / img.split('/')[-2] / img.split('/')[-1]
#         output_path.parent.mkdir(parents=True, exist_ok=True)
#         result.save(filename=str(output_path))

In [ ]:
# # check the ground truth for those images with detected objects
# image_ann_dir = dataset_dir / 'images_annotated'
# gt_output_dir = dataset_dir / 'images_predicted'/ 'gt'

# for img in images_detected_lvis:
#     obj = img.split('/')[-2]
#     name = img.split('/')[-1]
#     input_path = image_ann_dir / obj / name
#     output_path = gt_output_dir / obj / name
#     output_path.parent.mkdir(parents=True, exist_ok=True)
#     shutil.copy(input_path, output_path)

In [5]:
# get images_detected_lvis from already moved images

images_detected_lvis = []

for img in gt_output_dir.rglob("*.jpg"):
    img_path = str(img).replace(str(gt_output_dir), "./images")
    images_detected_lvis.append(img_path)

In [6]:
from ultralytics import YOLO

lp_weight_path = "runs/detect/n_orig_gpt/weights/best.pt"
yolo = YOLO(lp_weight_path)

for img in images_detected_lvis:
    input_path = dataset_dir / img
    result = yolo(input_path, verbose=False)[0]

    output_path = lp_output_dir / img.split("/")[-2] / img.split("/")[-1]
    output_path.parent.mkdir(parents=True, exist_ok=True)
    result.save(filename=str(output_path))

In [7]:
# mix the outputs into a new folder
mix_output_dir = dataset_dir / "images_predicted" / "mix"

for img in gt_output_dir.rglob("*.jpg"):
    new_name = img.stem + "_gt" + img.suffix
    new_img = mix_output_dir / img.parent.name / new_name
    new_img.parent.mkdir(parents=True, exist_ok=True)
    shutil.copy(img, new_img)

for img in lvis_output_dir.rglob("*.jpg"):
    new_name = img.stem + "_lvis" + img.suffix
    new_img = mix_output_dir / img.parent.name / new_name
    new_img.parent.mkdir(parents=True, exist_ok=True)
    shutil.copy(img, new_img)

for img in lp_output_dir.rglob("*.jpg"):
    new_name = img.stem + "_lp" + img.suffix
    new_img = mix_output_dir / img.parent.name / new_name
    new_img.parent.mkdir(parents=True, exist_ok=True)
    shutil.copy(img, new_img)